# Part 6: Board Test

This notebook is to run on the PYNQ! You'll need the bitfile and test dataset file from part 5.
We will load the bitfile we generated onto the PL of the PYNQ SoC. 

More details : https://pynq.readthedocs.io/en/latest/overlay_design_methodology/python_overlay_api.html

In [ ]:
from pynq import DefaultHierarchy, DefaultIP, allocate
from pynq import Overlay
from datetime import datetime
import pynq.lib.dma
import numpy as np


class NeuralNetworkOverlay(Overlay):
    def __init__(self, bitfile_name, dtbo=None, download=True, ignore_version=False, device=None):

        super().__init__(bitfile_name, dtbo=dtbo, download=download, ignore_version=ignore_version, device=device)

    def _print_dt(self, timea, timeb, N):
        dt = (timeb - timea)
        dts = dt.seconds + dt.microseconds * 10**-6
        rate = N / dts
        print("Classified {} samples in {} seconds ({} inferences / s)".format(N, dts, rate))
        return dts, rate
    def predict(self, X, y_shape, dtype=np.float32, debug=None, profile=False, encode=None, decode=None):
        if profile:
            timea = datetime.now()
        if encode is not None:
            X = encode(X)
        with allocate(shape=X.shape, dtype=dtype) as input_buffer, \
             allocate(shape=y_shape, dtype=dtype) as output_buffer:
            input_buffer[:] = X
            self.hier_0.axi_dma_0.sendchannel.transfer(input_buffer)
            self.hier_0.axi_dma_0.recvchannel.transfer(output_buffer)
            if debug:
                print("Transfer OK")
            self.hier_0.axi_dma_0.sendchannel.wait()
            if debug:
                print("Send OK")
            self.hier_0.axi_dma_0.recvchannel.wait()
            if debug:
                print("Receive OK")
            result = output_buffer.copy()
        if decode is not None:
            result = decode(result)
        if profile:
            timeb = datetime.now()
            dts, rate = self._print_dt(timea, timeb, len(X))
            return result, dts, rate
        return result


In [ ]:
if __name__ == '__main__':
    x_test = np.load('x_test.npy')
    y_test = np.load('y_test.npy')

    N = 100

    overlay = NeuralNetworkOverlay('design_1.bit')

    out = overlay.predict(x_test[:N],[N,10])
    predicted = out.argmax(axis=1)

    correct = (predicted == y_test[:N]).sum()
    accuracy = correct / N

    print("Model Accuracy: %.2f%%"%(accuracy*100))
